### Use case notebook: schedule CCB team meetings ###

In [ ]:
import os
import glob
import numpy as np

# There is a warning in the timeboard library that we want to suppress here
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
from pathlib import Path
import time
import timeboard as tb
import timeboard.calendars.US as US
import datetime
import holidays

# Appearance of the Notebook
from IPython.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Import the package
%load_ext autoreload
%autoreload 2
import cadence
from cadence.utils import FileOP
from cadence.mscheduler import Meetings
print(f'Package version: {cadence.__version__}')

### Meeting participants and settings ###

In [ ]:
# Local data
data_dir = os.path.join(os.environ.get('HOME'), 'data')
link = 'https://dsets.s3.amazonaws.com/ccb_names_list.csv'
team = FileOP().download_from_url(url=link, download_dir=data_dir)
tm_raw = pd.read_csv(ccb_names_file)
display(tm_raw)

### Cleanup of the spreadsheet ###
Just in case that was not done. To provide a consistent look.

In [ ]:
email_col = 'email'
name_col = 'name'
group_col = 'group'

# Lets do some cleanup of the spreadsheet
# small letters for the column names
tm = tm_raw.copy()
tm.columns = [col.lower() for col in tm.columns]

# small letters for emails and group, first letter capitalize for names
tm[email_col] = tm[email_col].str.lower()
tm[group_col] = tm[group_col].str.lower()
tm[name_col] = tm[name_col].str.title()

# Sort by group and then name
tm = tm.sort_values(by=['group', 'name'], ascending=True).reset_index(drop=True)

# Mark names that are not presenting
rm_name = ['nathan', 'ludwig']
rm_name = [nm.title() for nm in rm_name]
tm = tm.assign(presenting=True)
tm.loc[tm['name'].isin(rm_name), 'presenting'] = False

# save the name ist as a csv_file
presenter_list_file_name = f'{os.path.splitext(group_members_file_name)[0]}_list.csv'
presenter_list_file = os.path.join(data_dir, presenter_list_file_name)
tm.to_csv(presenter_list_file, index=False)
display(tm)

In [ ]:
# Remove the names that we do not want in the schedule
presenters_df = tm.loc[tm['presenting']==True]
display(presenters_df)

### Shuffle the data and create a presenter list ###

In [ ]:
# Let's do a random shuffling of the data frame
random_state = 984
presenters_df_shuffled = presenters_df.sample(frac=1, random_state=random_state)

# Get the names and the groups from the new data frame
name_list = list(presenters_df['name'].values)
group_list = list(presenters_df['group'].values)

# Instantiate the Meetings class with the list of names and groups from the data frame
meet = Meetings(name_list=name_list, group_list=group_list)

# We create a presenter list by merging the groups so that we have a member from a different group presenting each time
presenter_list = meet.create_name_sequence()
print(presenter_list)
display(meet.name_df)

### Manipulate the sequence of presenters ###

In [ ]:
presenter_list = list(meet.name_df['name'].values)
print(presenter_list)

# If we don't like this sequence, we can create a new one
new_presenter_list = ['Gerald', 'Grey', 'Sam', 'Tyrone', 'Sreya', 'Tram', 'Andreas', 'Anthony', 'Andrew']
print(new_presenter_list)

# We can replace the list of presenters with our new one. Just make sure that the names match.
new_presenter_list = meet.create_name_sequence(name_sequence=new_presenter_list, merge_groups=False)
print(new_presenter_list)

# We can see, that the internal data frame with the group associations has changed according to the new list
print()
display(meet.name_df)

### Create the meeting schedule ###

In [ ]:
start_date = '2024-08-28'
end_date = '2024-12-31'

cal = meet.create_timeboard(start_date=start_date, 
                            end_date=end_date,
                            start_name='Gerald')

skip_date = '2024-08-28'
skip_name = 'Everyone'
skip_comment = 'Repos, best practices discussion'

cal = meet.skip_date(cal_df=cal, 
                     date=skip_date, 
                     comment=skip_comment,
                     name=skip_name)

display(cal)

In [ ]:
# Save the presentation schedule
save_cols = ['date', 'name', 'comment']
schedule_name = f'ccb_presentations_2024.csv'
schedule_file = os.path.join(data_dir, schedule_name)
cal[save_cols].to_csv(schedule_file, index=False)
display(cal[save_cols])